# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

**Before running the program, we need to download the pre-trained detection model and put it in the current folder.** The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [ ]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [ ]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        ### "depth_image" changed to an instance variable
        depth_frame = frames.get_depth_frame()
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            self.depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [ ]:
import time
import math
from numpy import sign

class RobotController:
    def __init__(self):
        ### SET DEFUALT INSTANCE VARIABLES ###
        # self.d_list records data from previous detections
        self.d_list = []
        
        # self.lag is the length of time to look for detection within, in case of noise
        self.lag = 0.3
        
        # self.speed is the robot's speed
        self.speed = 1
        
        '''               mid      
        0  80  160  240   320  480  560  640
           wmin ws              we  wmax
        |   |   |    |     |    |   |   |
        [---+---+-------------------+---]       0
        |---+---+---------------+---+---|--hmin 60       
        |---+---+---------------+---+---|--hs   120
        |   |   |000000000000000|   |   |       180 
        |   |   |000000000000000|   |   |--he   240 mid
        |---+---+---------------+---+---|       300
        |---+---+---------------+---+---|       360
        [---+---+---------------+---+---]--hmax 420
        [---+---+---------------+---+---]       480
        '''
        self.hmid = 240
        self.wmid = 320

        self.wmin = 80 
        self.ws = 160
        self.we = 480
        self.wmax = 560

        self.hmin = 60
        self.hs = 120
        self.he = 180 #self.hmid
        self.hmax = self.hmid #360

        self.height = 640
        self.width = 480
        
        self.var = 50
        
    def SetVar(self, var):
        self.var = var

    
    def getDiag(self, bbox):
        return (((self.width*(bbox[2]-bbox[0]))**2+(self.height*(bbox[3]-bbox[1]))**2))**(1/2)

    def getMagnitude(self, b1w, b1h, b2w, b2h):
        return (((b2w-b1w)**2+(b2h-b1h)**2))**(1/2)
    
    def getDirection(self, b1w, b1h, b2w, b2h):
        return (math.tanh((b2h-b1h)/(b2w-b2h)))**(-1)
    
    
    def setTarget(self, tagetBox, currentBox):
        pass
    
    def stop(self):
        robot.stop()

    
    def follow(self, tagetBox, currentBox):
        centerW=320
        centerH=240
        # Set coordinates
        # target box
        tw1 = self.width * tagetBox[0]
        tw2 = self.width * tagetBox[2]

        th1 = self.height * tagetBox[1]
        th2 = self.height * tagetBox[3]

        tcw= (tw1+((tw2-tw1)/2))
        tch= (th1+((th2-th1)/2))

        tMagnitude = self.getMagnitude(centerW,centerH,tcw,tch)
        tDirection = self.getDirection(centerW,centerH,tcw,tch)
        """
        tQuadrant=1
        if tw1<320:
            if th1<=240:
                tQuadrant = 2
            else:
                tQuadrant = 3
        else:   
            if th1<=240:
                tQuadrant = 1
            else:
                tQuadrant = 4 """ 

        # current box
        cw1 = self.width * tagetBox[0]
        cw2 = self.width * tagetBox[2]

        ch1 = self.height * tagetBox[1]
        ch2 = self.height * tagetBox[3]
        
        ccw= (cw1+((cw2-cw1)/2))
        cch= (ch1+((ch2-ch1)/2))
        cMagnitude = self.getMagnitude(centerW,centerH,ccw,cch)
        cDirection = self.getDirection(centerW,centerH,ccw,cch)

        cQuadrant=0
        ch1+=self.var
        if ccw<280:
            if ch1<=180:
                cQuadrant = 3
            elif ch1>180 and ch1<300:
                cQuadrant = 4
            elif ch1>300:
                cQuadrant = 5
        elif ccw>=280 and ccw<=360:
                if ch1<=180:
                    cQuadrant = 2
                elif ch1>180 and ch1<300:
                    cQuadrant = 9
                elif ch1>300:
                    cQuadrant = 6 
        elif ccw>360:
                if ch1<=180:
                    cQuadrant = 1
                elif ch1>180 and ch1<300:
                    cQuadrant = 8
                elif ch1>300:
                    cQuadrant = 7 

        """magnitude = self.getMagnitude(tcw,tch,ccw,cch)
        direction = self.getDirection(tcw,tch,ccw,cch)"""
        # Determine if the current box is further from the center than the last box
        further=True
        if cMagnitude>tMagnitude:
            further=True
        else:
            further=False

        # Determine if the current box is further from the center than the last box
        speed=0.0
        minThreshold=20
        if cMagnitude>minThreshold:
            speed=0.3
        else:
            speed=0.5
        
        # forward backward
        # backward
        print ("quadrant", cQuadrant)
        if cQuadrant==1:
            # lostTarget = 1
            # move right -forward 
            speed=0.8
            robot.backward_left(speed)
            return
        
        if cQuadrant==2:
            # lostTarget = 1
            # move left-forward  
            speed=0.7
            robot.backward(speed)
            return

        if cQuadrant==3:
            # lostTarget = 1
            # move left-forwad 
            speed=0.8
            robot.backward_right(speed)
            return
        
        if cQuadrant==4:
            # lostTarget = 1
            # move right
            speed=0.65
            robot.left(speed)
            return
        if cQuadrant==5:
            # lostTarget = 1
            # move backward_left
            speed=0.8
            robot.forward_right(speed)
            return
        if cQuadrant==6:
            # lostTarget = 1
            # move backward
            speed=0.7
            robot.forward(speed)
            return
        if cQuadrant==7:
            # lostTarget = 1
            # move right-forward 
            speed=0.8
            robot.forward_left(speed)
            return
        if cQuadrant==8:
            # lostTarget = 1
            # move right-forward 
            speed=0.65
            robot.right(speed)
            return
            
        robot.stop()
        return
            

        

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=640, height=480)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

### Initialize the Robot Controller class
rc = RobotController()

#
# set initialized to false
initialized = False

# set initial coordinates to zero
tagetBox = None
tagetBoxDiag =0

def processing(change):
    global tagetBox,rc,tagetBoxDiag
    image = change['new']
     
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # reset biggest box coordinates
    biggestBox = None
    biggestBoxDiag = 5

    for det in matching_detections:
        bbox = det['bbox']
        if bbox[0] != None:
            diag = rc.getDiag(bbox)
        else:
            diag = 0
        if (diag > biggestBoxDiag):
            biggestBox = bbox
            biggestBoxDiag = diag
            if tagetBox ==None:
                tagetBox = biggestBox

    
    ### Pass detection to Robot Controller
    if biggestBoxDiag >10:
        rc.follow(tagetBox,biggestBox )
        tagetBox = biggestBox
        tagetBoxDiag = biggestBoxDiag

    else:
        if tagetBoxDiag >20:
            rc.follow(tagetBox,tagetBox )
        if biggestBoxDiag > tagetBoxDiag-20 or biggestBoxDiag< tagetBoxDiag+20:
            tagetBox = biggestBox
            tagetBoxDiag = biggestBoxDiag
        rc.stop()

    
    for det in matching_detections:
        bbox = det['bbox']
        #cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

In [ ]:
rc.SetVar(0)

## Tasks

### 1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to tutorial 2 to get the depth information for a specific point)
### 2. Please try to add collision avoidance function into this program to protect the robot. (Note: You can refer to tutorial 4 for the collision avoidance)
### 3. Think about how to control the robot to move towards the detected human